In [1]:
import os, sys
import argparse
import numpy as np
from collections import defaultdict
import json
import time
import multiprocessing
import copy
import os.path as osp
# from utils import IdGenerator, id2rgb
import pdb
import torch
try:
    import PIL.Image     as Image
except:
    print("Failed to import the image processing packages.")
    sys.exit(-1)
from pycocotools.coco import COCO
import numpy as np
# import skimage.io as io
import pylab

In [13]:
stage='val'
b = 110
step = 160
save_path = './datasets/lvis/lvis_trainval_1230'

In [5]:
inst_gt_json_file = "./datasets/lvis/lvis_v0.5_"+stage+".json"
data_path = './datasets/lvis/images/'+stage+'2017'
with open(inst_gt_json_file, 'r') as f:
    inst_gt = json.load(f)

In [6]:
sorted_cls_id_file = os.path.join(save_path, 'lvis_sorted_id_all.json')
with open(sorted_cls_id_file, 'r') as f:
    sorted_cls_id = json.load(f)

In [8]:
sorted_class_ids_top_b = sorted_cls_id[:b]

In [11]:
json.dump(sorted_class_ids_top_b, open(os.path.join(save_path, 'lvis_sorted_id_top'+str(b)+'.json'), 'w'))

In [12]:
import torch
import torchvision


min_keypoints_per_image = 10


def _count_visible_keypoints(anno):
    return sum(sum(1 for v in ann["keypoints"][2::3] if v > 0) for ann in anno)


def _has_only_empty_bbox(anno):
    return all(any(o <= 1 for o in obj["bbox"][2:]) for obj in anno)


def has_valid_annotation(anno):
    # if it's empty, there is no annotation
    if len(anno) == 0:
        return False
    # if all boxes have close to zero area, there is no annotation
    if _has_only_empty_bbox(anno):
        return False
    # keypoints task have a slight different critera for considering
    # if an annotation is valid
    if "keypoints" not in anno[0]:
        return True
    # for keypoint detection tasks, only consider valid images those
    # containing at least min_keypoints_per_image
    if _count_visible_keypoints(anno) >= min_keypoints_per_image:
        return True
    return False

class COCODataset(torchvision.datasets.coco.CocoDetection):
    def __init__(self, root, ann_file, remove_images_without_annotations=False):
        super(COCODataset, self).__init__(root, ann_file)
        self.ids = sorted(self.ids)

        # filter images without detection annotations
        if remove_images_without_annotations:
            ids = []
            for img_id in self.ids:
#                 ann_ids = self.coco.getAnnIds(imgIds=img_id, catIds = id_gt,iscrowd=None)
                ann_ids = self.coco.getAnnIds(imgIds=img_id, iscrowd=None)

                anno = self.coco.loadAnns(ann_ids)
                if has_valid_annotation(anno):
                    ids.append(img_id)
            self.ids = ids

        self.categories = {cat['id']: cat['name'] for cat in self.coco.cats.values()}

#         self.json_category_id_to_contiguous_id = {
#             v: i + 1 for i, v in enumerate(self.coco.getCatIds())
#         }
#         self.contiguous_category_id_to_json_id = {
#             v: k for k, v in self.json_category_id_to_contiguous_id.items()
#         }
        self.id_to_img_map = {k: v for k, v in enumerate(self.ids)}
#         self._transforms = transforms
        
    def __getitem__(self, idx):
        img, anno = super(COCODataset, self).__getitem__(idx)
        return img, anno
    
    def get_img_info(self, index):
        img_id = self.id_to_img_map[index]
        img_data = self.coco.imgs[img_id]
        return img_data
    
coco = COCODataset(data_path,inst_gt_json_file, True)

loading annotations into memory...
Done (t=1.78s)
creating index...
index created!


In [16]:
#构建只有108类所有标注的小数据集
inst_gt_subset = inst_gt.copy()
annotations_subset = []

for class_i in sorted_class_ids_top_b:
    ann_list = coco.coco.getAnnIds(catIds=class_i)
    annotations_subset.extend(coco.coco.loadAnns(ids=ann_list))
inst_gt_subset['annotations'] = annotations_subset

json.dump(inst_gt_subset, open(os.path.join(save_path, 'lvis_v0.5_'+stage+'_top'+str(b)+'.json'), 'w'))


In [17]:
if stage == 'val':
    for i in inst_gt_subset['categories']:
        if i['id'] in sorted_cls_id[:b]:
            i['step_state'] = 'b0'
       elif i['id'] in sorted_cls_id[b:270+160]:
            i['step_state'] = 't0'
       else:
            i['step_state'] = 't1'
    json.dump(inst_gt_subset, open(os.path.join(save_path, 'lvis_v0.5_'+stage+'_top'+str(b)+'.json'), 'w'))

